# Lecture 5: Features and Matching
### Juan Carlos Niebles and Adrien Gaidon


In [ ]:
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
from skimage import feature

## Contents

1. RANSAC
2. Harris corner detector

## Line detection

Let's suppose we want to find lines on this image. One way to do that, is to first run Canny edge detection to get a list of points, and then run a line estimation algorithm. Let's do Canny first:

In [ ]:
img = io.imread("field.jpg", as_gray=True)
edge = feature.canny(img, sigma=4)
plt.figure(figsize=(18,10))
plt.subplot(1,2,1); plt.imshow(img, cmap="gray"); plt.title("input image", fontsize=20);
plt.subplot(1,2,2); plt.imshow(edge, cmap="gray"); plt.title("edge map", fontsize=20);

Now, let's get the coordinates of all edge points:

In [ ]:
y, x = np.nonzero(edge)
print(x[0:10], x[-10:])
print(y[0:10], y[-10:])

plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.imshow(img, cmap="gray"); plt.title("input image", fontsize=20);
plt.subplot(1,2,2); plt.imshow(edge, cmap="gray"); plt.title("edge map", fontsize=20);
plt.tight_layout();plt.show()

## Line detection with Least Squares fitting?



In [ ]:
reg = LinearRegression().fit(x.reshape(-1,1), y)
print(f"Line equation: y = {reg.coef_[0]:.2f}*x + {reg.intercept_:.2f}")
x_line = np.linspace(0,img.shape[1]-1).reshape(-1,1)
y_line = reg.predict(x_line)

plt.figure(figsize=(20,10))
plt.subplot(1,3,1); plt.imshow(img, cmap="gray"); plt.title("input image", fontsize=20);
plt.subplot(1,3,2); plt.imshow(edge, cmap="gray"); plt.title("edge map", fontsize=20);
plt.subplot(1,3,3); plt.imshow(edge, cmap="gray"); plt.title("least squares line", fontsize=20);
plt.plot(x_line, y_line, 'r')
plt.tight_layout()

In [ ]:
img = io.imread("road.jpg", as_gray=True)
edge = feature.canny(img, sigma=5)
y, x = np.nonzero(edge)
reg = LinearRegression().fit(x.reshape(-1,1), y)
print(f"Line equation: y = {reg.coef_[0]:.2f}*x + {reg.intercept_:.2f}")

x_line = np.linspace(0,img.shape[1]-1).reshape(-1,1)
y_line = reg.predict(x_line)

plt.figure(figsize=(20,10))
plt.subplot(1,3,1); plt.imshow(img, cmap="gray"); plt.title("input image", fontsize=20);
plt.subplot(1,3,2); plt.imshow(edge, cmap="gray"); plt.title("edge map", fontsize=20);
plt.subplot(1,3,3); plt.imshow(edge, cmap="gray"); plt.title("least squares line", fontsize=20);
plt.plot(x_line, y_line, 'r')
plt.tight_layout()

## Problems:
1. How do we deal with noise/non-relevant edge points?
2. How many lines?

Both Line detection with Hough transform and line detection with RANSAC can solve these issues.


## Line detection with RANSAC


In [ ]:
ransac = RANSACRegressor(base_estimator=LinearRegression())
ransac.fit(x.reshape(-1,1), y)
print(f"Line equation: y = {ransac.estimator_.coef_[0]:.2f}*x + {ransac.estimator_.intercept_:.2f}")
x_line = np.linspace(0, img.shape[1]-1).reshape(-1,1)
y_line = ransac.predict(x_line)

plt.figure(figsize=(20,10))
plt.subplot(1,3,1); plt.imshow(img, cmap="gray"); plt.title("input image", fontsize=20);
plt.subplot(1,3,2); plt.imshow(edge, cmap="gray"); plt.title("edge map", fontsize=20);
plt.subplot(1,3,3); plt.plot(x_line, y_line, 'r--'); plt.imshow(edge, cmap="gray"); plt.title("line with ransac", fontsize=20)
plt.tight_layout()

## Line detection with RANSAC


RANSAC decides which edge points belong to the line (__inliers__), and which are noise (__outliers__). Not what we expected?

In [ ]:
x_inliers = x[np.nonzero(ransac.inlier_mask_)]
y_inliers = y[np.nonzero(ransac.inlier_mask_)]
print(f"Found {x_inliers.size} inliers.")
plt.figure(figsize=(20,10))
plt.subplot(1,3,1); plt.imshow(edge, cmap="gray"); plt.title("edge map", fontsize=20);
plt.subplot(1,3,2); plt.plot(x_line, y_line, 'r--'); plt.imshow(edge, cmap="gray"); plt.title("line with ransac", fontsize=20)
plt.subplot(1,3,3); plt.plot(x_inliers, y_inliers, 'g.'); plt.imshow(edge, cmap="gray"); plt.title("inliers", fontsize=20)
plt.tight_layout()

## Line detection with RANSAC
How do we detect other lines in the image? Ideas?

In [ ]:
def get_next_line(x, y, img, edge, ransac):
    pass
    return x, y, img, edge, ransac

In [ ]:
x, y, img, edge, ransac = get_next_line(x, y, img, edge, ransac)

## RANSAC goes beyond line detection

The examples we have seen with line detection are rather simple illustrations for the use of RANSAC. However, we can use RANSAC to fit more complex models. Another example in an upcoming homework.

## Harris corner detector


In [ ]:
img = io.imread("board.jpg", as_gray=True)
plt.imshow(img, cmap="gray")

## Harris corner detector

Corner response function. Uses the matrix $M$ to calculate $\theta$ at each pixel. See lecture slides.

In [ ]:
response = corner_harris(img, sigma=1)
plt.figure(figsize=(20,10))
plt.subplot(1,2,1); plt.imshow(img, cmap="gray")
plt.subplot(1,2,2); plt.imshow(response, cmap="jet")
plt.tight_layout()

## Harris corner detection

In [ ]:
coords = corner_peaks(response, min_distance=3, threshold_rel=0.02)                   
                      
plt.figure(figsize=(30,15))
plt.subplot(1,3,1); plt.imshow(img, cmap="gray"); plt.title("input", fontsize=30)
plt.subplot(1,3,2); plt.imshow(response, cmap="jet"); plt.title("corner response function", fontsize=30)
plt.subplot(1,3,3); plt.plot(coords[:,1], coords[:,0],'r.'); plt.imshow(response, cmap="gray"); plt.title("peak locations", fontsize=30)
plt.tight_layout()